<a href="https://colab.research.google.com/github/mg-regina/Programacion/blob/main/PROYECTO_NUM_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Martínez Garduño Regina
# ¿Qué se importó?
import numpy as np
import pandas as pd
from google.colab import drive

# Definimos nuestras clases
class Grafica:
    """Esto es una gráfica:)."""
    def __init__(self, nodos, arcos):
        self.matriz = np.zeros((len(nodos), len(nodos)))
        for x in arcos:
            self.matriz[x[0], x[1]] = 1

    def __str__(self):
        return str(self.matriz)

class Red(Grafica):
    """Esto es una clase grafica, la cual heredará la clase grafica:)."""
    def __init__(self, nodos, arcos):
        super().__init__(nodos, arcos)
        self.matriz_probabilidades = np.zeros_like(self.matriz)

    def calcular_matriz_probabilidades(self):
        for i in range(len(self.matriz)):
            suma = self.matriz[i, :].sum()
            if suma > 0:
                self.matriz_probabilidades[i, :] = self.matriz[i, :] / suma
            else:
                self.matriz_probabilidades[i, :] = np.zeros(len(self.matriz))

    def calcular_pagerank(self, d=0.85, tol=1e-6):
        """Aquí se calculará el PangeRank utilizando el "damping factor". """
        n = len(self.matriz)
        self.calcular_matriz_probabilidades()
        U = np.ones((n, n)) / n
        P = d * self.matriz_probabilidades + (1 - d) * U
        pi = np.ones((1, n)) / n
        pi_1 = pi @ P

        while np.linalg.norm(pi - pi_1) > tol:
            pi = pi_1
            pi_1 = pi @ P

        return pi_1.flatten()

    def calcular_pagerank_ru(self, indices_ru, d=0.85, tol=1e-6, max_iter=100):
        """Ahora, se calculará el PageRank PERO de las pagínas con terminación .ru"""
        n = len(self.matriz)
        if self.matriz_probabilidades is None:
            self.calcular_matriz_probabilidades()

        matriz_uniforme = np.ones((n, n)) / n
        matriz_transicion = d * self.matriz_probabilidades + (1 - d) * matriz_uniforme
        pagerank = np.zeros(n)
        for idx in indices_ru:
            pagerank[idx] = 1 / len(indices_ru)
        for _ in range(max_iter):
            nuevo_pagerank = matriz_transicion.T @ pagerank
            if np.linalg.norm(nuevo_pagerank - pagerank, 1) < tol:
                return nuevo_pagerank
            pagerank = nuevo_pagerank

        return pagerank

def crear_red_desde_excel(ruta_archivo):
    """Construirá un objeto Red, dado un archivo de Excel"""
    datos = pd.read_excel(ruta_archivo)
    nodos = list(datos["Index"] - 1)  # Convertir índices a base 0
    enlaces = []
    for idx, fila in datos.iterrows():
        citados = str(fila["Cited by"]).split(',')
        for citado in citados:
            citado = citado.strip()
            if citado.isdigit():
                enlaces.append((int(citado) - 1, idx))  # Base 0 para ambos índices

    return Red(nodos, enlaces), datos

# Una vez definidas nuestras clases, pasamos a implementar un  "__main__"
if __name__ == "__main__":
    # Google Drive
    drive.mount('/content/drive', force_remount=True)
    ruta_excel = "/content/drive/MyDrive/Web.xlsx"

    red, df = crear_red_desde_excel(ruta_excel)

   # Finalmente, responderemos las preguntas
 # Pregunta I: ¿Cuál es la página web más importante...?
    pagerank_resultado = red.calcular_pagerank()
    pagina_importante = np.argmax(pagerank_resultado) + 1

    print("Resultados de PageRank estándar (Pregunta 1):", pagerank_resultado)
    print("Página más importante (Pregunta 1):", pagina_importante)

    # Pregunta II: Defina ahora un vector π el cúal tendrá la terminación .ru
if "Website" in df.columns:
    indices_ru = [
        i for i, url in enumerate(df["Website"])
        if isinstance(url, str) and url.lower().strip().endswith(".ru")
    ]

    if indices_ru:
        try:
            pagerank_ru_resultado = red.calcular_pagerank_ru(indices_ru)
            pagina_importante_ru = np.argmax(pagerank_ru_resultado) + 1
            print("\nResultados de PageRank con páginas .ru (Pregunta 2):", pagerank_ru_resultado)
            print("Página más importante con .ru (Pregunta 2):", pagina_importante_ru)
        except Exception as e:
            print("\nError al calcular el PageRank para páginas .ru:", str(e))
    else:
        print("\nNo se encontraron páginas con terminación '.ru' en los datos proporcionados (Pregunta 2).")
else:
    print("\nError: La columna 'Website' no existe en los datos (Pregunta 2).")


# Pregunta III: Utilice los valores 0.5,0.85,1 para d, calcular el PageRank
valores_d = [0.5, 0.85, 1.0]  # Valores d
print("\nResultados para diferentes valores de d (Pregunta 3):")

for d in valores_d:
    try:
        pagerank_modificado = red.calcular_pagerank(d=d)
        pagina_importante = np.argmax(pagerank_modificado) + 1
        print(f"\nDamping factor (d) = {d}:")
        print("Vector de PageRank:", pagerank_modificado)
        print("Página más importante:", pagina_importante)
    except Exception as e:

        print(f"\nError al calcular PageRank con d = {d}: {str(e)}")

Mounted at /content/drive
Resultados de PageRank estándar (Pregunta 1): [4.36095886e-06 5.71216390e-06 4.15619363e-06 7.63861952e-06
 3.75063521e-06 4.93162098e-06 7.20351852e-06 5.05091425e-06
 6.13151513e-06 5.29650322e-06 3.45797359e-06 3.61858330e-06
 4.68195088e-06 4.60022938e-06 5.85797202e-06 7.17214702e-06
 7.22936600e-06 7.15264950e-06 7.18266617e-06 9.27941187e-06
 7.18266617e-06 7.14311332e-06 7.08846838e-06 9.28740993e-06
 7.12996108e-06 5.55837256e-06]
Página más importante (Pregunta 1): 24

Resultados de PageRank con páginas .ru (Pregunta 2): [0.00159979 0.00209547 0.00152467 0.00280218 0.0013759  0.00180914
 0.00264257 0.0018529  0.00224931 0.00194299 0.00126854 0.00132746
 0.00171755 0.00168757 0.00214896 0.00263106 0.00265205 0.00262391
 0.00263492 0.0034041  0.00263492 0.00262041 0.00260036 0.00340703
 0.00261558 0.00203906]
Página más importante con .ru (Pregunta 2): 24

Resultados para diferentes valores de d (Pregunta 3):

Damping factor (d) = 0.5:
Vector de PageRa